In [98]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
import os
import stdatamodels.jwst.datamodels as dm
from astropy.io import fits
from glob import glob
import pandas as pd

In [97]:
class DataCube:
	"""
	An object combining multiple 1D and 2D spectra from a given folder.
	
	
	Parameters :
	----------
	folder :
		path to the folder containing the final data. Will loop on every fits file 
		and keep the one containing the _s2d or _x1d suffix
	
	Properties : 
	----------
	table : 
		a table used for making a correspondence between a source id, a s2d file and a x1d file. 
		Those files are stored as a list of paths 
	"""
	
	def __init__(self, folder):	
		s2dList = [[file] for file in sorted(glob(os.path.join(folder, '*_s2d.fits')))]
		x1dList = [[x] if os.path.exists(x := file[0].replace("_s2d", "_x1d")) else [None] for file in s2dList]
		sourceList = [fits.open(file[0])[1].header["SOURCEID"] for file in s2dList]
		
		self.table = pd.DataFrame({"sourceID": sourceList, "s2d": s2dList, "x1d": x1dList})
		
		# Initializes the dataframe which will contain the data models
		self.dataTable = pd.DataFrame({"sourceID" : [], "s2d" : [], "x1d" : []})

		
		
	def combineDataCube(self, datacube):
		"""
		Combines 2 Datacubes
		Parameters
		----------
		datacube :
			Another datacube to be appended to this one
		"""
		
		print("Starting Combining Datacubes")
		
		print("Merging...")
		# Perform an outer join to include all sourceIDs from both DataFrames
		merged = self.table.merge(
			datacube.table, on="sourceID", how="outer", suffixes=("_self", "_other")
		)
		
		print("Replacing empty values...")
		# Fill missing values in 's2d' and 'x1d' columns with empty lists or zero, as appropriate
		merged["s2d_self"] = merged["s2d_self"].apply(lambda x: x if isinstance(x, list) else [])
		merged["s2d_other"] = merged["s2d_other"].apply(lambda x: x if isinstance(x, list) else [])
		merged["x1d_self"] = merged["x1d_self"].apply(lambda x: x if isinstance(x, list) else [])
		merged["x1d_other"] = merged["x1d_other"].apply(lambda x: x if isinstance(x, list) else [])
		
		print("Appending paths...")
		# Combine the 's2d' and 'x1d' columns
		merged["s2d"] = merged["s2d_self"] + merged["s2d_other"]
		merged["x1d"] = merged["x1d_self"] + merged["x1d_other"]
		
		# Keep only necessary columns: 'sourceID', 's2d', 'x1d'
		self.table = merged[["sourceID", "s2d", "x1d"]]
		
		print("Finished Combining Datacubes!")

	def preloadDataCube(self):
		"""
		Initializes self.dataTable, a table structurally identical to self.table, 
		except the paths are replaced by the corresponding datamodels
		"""
		print("Starting loading data...")
		print("Copying...")
		self.dataTable = self.table.copy()
		
		# Process lists of file paths
		def processList(file_list):
			return [dm.open(file) for file in file_list if isinstance(file, str)]
		
		print("Loading...")
		# Process the 'x1d' and 's2d' columns
		self.dataTable["x1d"] = self.dataTable["x1d"].apply(processList)
		self.dataTable["s2d"] = self.dataTable["s2d"].apply(processList)
		
		print("Finished loading data!")
		
	def exploreDataCube(self):
		fig, axes = plt.subplots(4, 1, figsize=(18, 7), gridspec_kw={'hspace': 0})

In [69]:
_ = DataCube("/home/tim-dewachter/Documents/Thèse/BetterNIRSpecBackground/mastDownload/JWST/CEERS-NIRSPEC-P5-PRISM-MSATA/Final")
_.table

,sourceID,s2d,x1d
0,44419,[/home/tim-dewachter/Documents/Thèse/BetterNIR...,[/home/tim-dewachter/Documents/Thèse/BetterNIR...
